In [1]:
from scipy.integrate import solve_ivp
import numpy as np
import pandas as pd
from scipy.optimize import minimize_scalar

In [2]:
def fn(t, y, pars):
    un, tf, tt = y
    ga, risk, ppv = pars['ga'], pars['risk'], pars['ppv']
    r = ga * risk / (1 - risk) / ppv
    
    return np.array([
    - r * un                                   + ga * (tf + tt),
      r * (1 - ppv) *  un       - r * ppv * tf - ga * tf,
      r *       ppv * (un + tf)                - ga * tt
    ])

In [3]:
ppv = 0.6

def target(x, ppv):
    p = {
        'ga': 0.02,
        'risk': x,
        'ppv': ppv
    }

    sol = solve_ivp(fn, [0, 1000], [1, 0, 0], args = (p, ))

    y1 = sol.y[:, -1]
    pi = y1[-1] / y1[1:].sum()

    return (pi / ppv - 1.8) ** 2

def sim(x, ppv):
    p = {
        'ga': 0.02,
        'risk': x,
        'ppv': ppv
    }

    sol = solve_ivp(fn, [0, 1000], [1, 0, 0], args = (p, ))

    y1 = sol.y[:, -1]
    p['pi'] = y1[-1] / y1[1:].sum()

    return p

In [4]:
ppv = 0.2

opt = minimize_scalar(target, bounds = [0.01, 0.99], args = (ppv, ))

sim(opt.x, ppv)

{'ga': 0.02, 'risk': 0.1999602846488997, 'ppv': 0.2, 'pi': 0.3599998307372604}

In [5]:
ms = list()
for ppv in np.linspace(0.01, 0.55, 100):
    opt = minimize_scalar(target, bounds = [0.01, 0.99], args = (ppv, ))
    ms.append(sim(opt.x, ppv))

ms = pd.DataFrame(ms)

In [7]:
ms.to_csv('out/sim_ppv_treated.csv')